In [7]:
import random, math
import copy
import sys
import math

A = np.array([[ 3,  8,  1,  2], [ 3, -2,  5,  4], [ 4, -3,  7,  0],  [ 4,  2, -2,  0]])
b= np.array([-265, 1177, 1162, -164])
# x = [52 -87  99  88] # Solution vector
# xi = qi1 + 2qi2 + 4qi3 - 8qi4
# Subrange interval = 16

T = np.array([48, -80,  96,  96])
# q = ([0, 0, 1, 0], [1, 0, 0, 1]. [1. 1, 0, 0]. [0, 0, 0, 1])
# Please read the minimum energy carefully.
# We will updae q1 to q01 at the next version

c = np.int64(b - np.dot(A,T))
(Dimension, Dimension_same) = A.shape
qubits = 3

QM = np.zeros(((qubits+1)*Dimension, (qubits+1)*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        cef1 = pow(2,2*qubits)*pow(A[k][i],2)
        cef2 = pow(2,qubits+1)*A[k][i]*c[k]
        po2 = (qubits+1)*i + qubits 
        QM[po2][po2] = QM[po2][po2] + cef1 + cef2
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po1 = (qubits+1)*i + l
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l in range (qubits):
            qcef = pow(2, l+qubits+1)*pow(A[k][i],2)
            po3 = (qubits+1)*i + l
            po4 = (qubits+1)*i + qubits 
            QM[po3][po4] = QM[po3][po4] - qcef
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = (qubits+1)*i + l1
                po2 = (qubits+1)*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                
### Second quadratic term ###                 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            qcef = pow(2, 2*qubits+1)*A[k][i]*A[k][j]
            po3 = (qubits+1)*i + qubits
            po4 = (qubits+1)*j + qubits
            QM[po3][po4] = QM[po3][po4] + qcef
            for l in range (qubits):
                qcef = pow(2, l+qubits+1)*A[k][i]*A[k][j]
                po5 = (qubits+1)*i + qubits 
                po6 = (qubits+1)*j + l
                QM[po5][po6] = QM[po5][po6] - qcef
                po7 = (qubits+1)*i + l
                po8 = (qubits+1)*j + qubits
                QM[po7][po8] = QM[po7][po8] - qcef
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = (qubits+1)*i + l1
                    po2 = (qubits+1)*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range((qubits+1)*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",(qubits+1)*Dimension,"','q",(qubits+1)*Dimension,"'):",format(QM[(qubits+1)*Dimension-1][(qubits+1)*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range((qubits+1)*Dimension-1):
    for j in range(i+1,(qubits+1)*Dimension):
        qdrt1 = i + 1
        qdrt2 = j + 1
        if i == (qubits+1)*Dimension-2 and j == (qubits+1)*Dimension-1:
            print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
        else:
            print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

# Matrix Q is
[[  -94.   200.   400.  -800.    28.    56.   112.  -224.    76.   152.
    304.  -608.    36.    72.   144.  -288.]
 [    0.   -88.   800. -1600.    56.   112.   224.  -448.   152.   304.
    608. -1216.    72.   144.   288.  -576.]
 [    0.     0.   224. -3200.   112.   224.   448.  -896.   304.   608.
   1216. -2432.   144.   288.   576. -1152.]
 [    0.     0.     0.  4352.  -224.  -448.  -896.  1792.  -608. -1216.
  -2432.  4864.  -288.  -576. -1152.  2304.]
 [    0.     0.     0.     0.  1393.   324.   648. -1296.   -54.  -108.
   -216.   432.    16.    32.    64.  -128.]
 [    0.     0.     0.     0.     0.  2948.  1296. -2592.  -108.  -216.
   -432.   864.    32.    64.   128.  -256.]
 [    0.     0.     0.     0.     0.     0.  6544. -5184.  -216.  -432.
   -864.  1728.    64.   128.   256.  -512.]
 [    0.     0.     0.     0.     0.     0.     0. -5312.   432.   864.
   1728. -3456.  -128.  -256.  -512.  1024.]
 [    0.     0.     0.     0.     0.     0.     0.

In [8]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-94.0, ('q2','q2'):-88.0, ('q3','q3'):224.0, ('q4','q4'):4352.0, ('q5','q5'):1393.0, ('q6','q6'):2948.0, ('q7','q7'):6544.0, ('q8','q8'):-5312.0, ('q9','q9'):-725.0, ('q10','q10'):-1292.0, ('q11','q11'):-1952.0, ('q12','q12'):11488.0, ('q13','q13'):176.0, ('q14','q14'):392.0, ('q15','q15'):944.0, ('q16','q16'):32.0}

quadratic = {('q1','q2'):200.0, ('q1','q3'):400.0, ('q1','q4'):-800.0, ('q1','q5'):28.0, ('q1','q6'):56.0, ('q1','q7'):112.0, ('q1','q8'):-224.0, ('q1','q9'):76.0, ('q1','q10'):152.0, ('q1','q11'):304.0, ('q1','q12'):-608.0, ('q1','q13'):36.0, ('q1','q14'):72.0, ('q1','q15'):144.0, ('q1','q16'):-288.0, ('q2','q3'):800.0, ('q2','q4'):-1600.0, ('q2','q5'):56.0, ('q2','q6'):112.0, ('q2','q7'):224.0, ('q2','q8'):-448.0, ('q2','q9'):152.0, ('q2','q10'):304.0, ('q2','q11'):608.0, ('q2','q12'):-1216.0, ('q2','q13'):72.0, ('q2','q14'):144.0, ('q2','q15'):288.0, ('q2','q16'):-576.0, ('q3','q4'):-3200.0, ('q3','q5'):112.0, ('q3','q6'):224.0, ('q3','q7'):448.0, ('q3','q8'):-896.0, ('q3','q9'):304.0, ('q3','q10'):608.0, ('q3','q11'):1216.0, ('q3','q12'):-2432.0, ('q3','q13'):144.0, ('q3','q14'):288.0, ('q3','q15'):576.0, ('q3','q16'):-1152.0, ('q4','q5'):-224.0, ('q4','q6'):-448.0, ('q4','q7'):-896.0, ('q4','q8'):1792.0, ('q4','q9'):-608.0, ('q4','q10'):-1216.0, ('q4','q11'):-2432.0, ('q4','q12'):4864.0, ('q4','q13'):-288.0, ('q4','q14'):-576.0, ('q4','q15'):-1152.0, ('q4','q16'):2304.0, ('q5','q6'):324.0, ('q5','q7'):648.0, ('q5','q8'):-1296.0, ('q5','q9'):-54.0, ('q5','q10'):-108.0, ('q5','q11'):-216.0, ('q5','q12'):432.0, ('q5','q13'):16.0, ('q5','q14'):32.0, ('q5','q15'):64.0, ('q5','q16'):-128.0, ('q6','q7'):1296.0, ('q6','q8'):-2592.0, ('q6','q9'):-108.0, ('q6','q10'):-216.0, ('q6','q11'):-432.0, ('q6','q12'):864.0, ('q6','q13'):32.0, ('q6','q14'):64.0, ('q6','q15'):128.0, ('q6','q16'):-256.0, ('q7','q8'):-5184.0, ('q7','q9'):-216.0, ('q7','q10'):-432.0, ('q7','q11'):-864.0, ('q7','q12'):1728.0, ('q7','q13'):64.0, ('q7','q14'):128.0, ('q7','q15'):256.0, ('q7','q16'):-512.0, ('q8','q9'):432.0, ('q8','q10'):864.0, ('q8','q11'):1728.0, ('q8','q12'):-3456.0, ('q8','q13'):-128.0, ('q8','q14'):-256.0, ('q8','q15'):-512.0, ('q8','q16'):1024.0, ('q9','q10'):316.0, ('q9','q11'):632.0, ('q9','q12'):-1264.0, ('q9','q13'):44.0, ('q9','q14'):88.0, ('q9','q15'):176.0, ('q9','q16'):-352.0, ('q10','q11'):1264.0, ('q10','q12'):-2528.0, ('q10','q13'):88.0, ('q10','q14'):176.0, ('q10','q15'):352.0, ('q10','q16'):-704.0, ('q11','q12'):-5056.0, ('q11','q13'):176.0, ('q11','q14'):352.0, ('q11','q15'):704.0, ('q11','q16'):-1408.0, ('q12','q13'):-352.0, ('q12','q14'):-704.0, ('q12','q15'):-1408.0, ('q12','q16'):2816.0, ('q13','q14'):80.0, ('q13','q15'):160.0, ('q13','q16'):-320.0, ('q14','q15'):320.0, ('q14','q16'):-640.0, ('q15','q16'):-1280.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q2 q3 q4 q5 q6 q7 q8 q9  energy num_oc. ...
39   1   1   0   0   0   0   0   1  1  0  0  1  0  0  1  1 -6660.0       1 ...
40   1   0   1   0   0   0   0   1  1  0  0  0  1  0  1  0 -6658.0       2 ...
343  1   0   1   0   0   0   0   1  1  0  0  1  0  0  1  0 -6657.0       1 ...
43   1   1   0   0   1   0   0   1  0  1  0  1  0  0  1  0 -6645.0       1 ...
49   1   0   1   0   1   0   0   1  1  0  0  0  1  0  1  0 -6614.0       3 ...
50   0   0   1   0   0   0   0   1  1  0  0  0  1  0  1  0 -6612.0       1 ...
53   1   1   0   0   1   0   0   1  1  0  0  0  1  0  1  1 -6607.0       1 ...
55   1   1   0   0   1   0   0   1  0  1  0  1  0  0  1  1 -6604.0       1 ...
56   0   0   1   0   1   0   0   1  1  0  0  0  1  0  1  0 -6604.0       2 ...
59   0   0   1   0   1   0   0   1  1  0  0  1  0  0  1  0 -6591.0       2 ...
62   0   0   1   0   0   0   0   1  1  0  0  1  0  0  1  0 -6583.0       1 ...
63   0   0   1   0   0   0   0   1  1  0  0  0  1  0

In [9]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q2 q3 q4 q5 q6 q7 q8 q9  energy num_oc. ...
53   1   1   0   0   0   0   0   1  0  1  0  1  0  0  1  1 -6660.0       1 ...
55   1   0   1   0   0   0   0   1  1  0  0  0  1  0  1  0 -6658.0       1 ...
56   0   1   0   0   1   0   0   1  0  1  0  1  0  0  1  0 -6655.0       1 ...
59   0   1   0   0   0   1   0   1  0  1  0  0  0  0  1  0 -6644.0       1 ...
60   0   0   1   0   0   0   0   1  0  1  0  1  0  0  1  0 -6631.0       1 ...
61   0   1   0   0   0   1   0   1  0  1  0  1  0  0  1  1 -6630.0       1 ...
62   0   1   0   0   0   0   0   1  0  1  0  0  0  0  1  1 -6629.0       1 ...
64   0   0   1   0   1   0   0   1  1  0  0  0  1  0  1  0 -6604.0       1 ...
65   0   1   0   0   0   0   0   1  0  1  0  0  0  0  1  0 -6604.0       2 ...
66   0   1   0   0   1   1   0   1  0  1  0  0  0  0  1  0 -6604.0       1 ...
72   0   1   0   0   1   0   0   1  0  1  0  0  1  0  1  1 -6593.0       1 ...
75   1   1   0   0   0   1   0   1  1  0  0  1  0  0

In [10]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q2 q3 q4 q5 q6 q7 q8 q9  energy num_oc. ...
11   0   1   0   0   1   0   0   1  0  1  0  1  0  0  1  1 -6690.0       3 ...
288  0   1   0   0   1   0   0   1  0  1  0  1  0  0  1  1 -6690.0       1 ...
12   1   1   0   0   1   0   0   1  1  0  0  1  0  0  1  1 -6676.0       1 ...
15   1   1   0   0   1   0   0   1  0  1  0  0  0  0  1  0 -6662.0       2 ...
16   1   1   0   0   0   0   0   1  0  1  0  1  0  0  1  1 -6660.0       2 ...
17   1   1   0   0   0   0   0   1  0  1  0  1  0  0  1  0 -6657.0       1 ...
18   1   0   1   0   0   0   0   1  1  0  0  1  0  0  1  0 -6657.0       1 ...
19   0   1   0   0   1   0   0   1  0  1  0  1  0  0  1  0 -6655.0       7 ...
20   1   1   0   0   1   0   0   1  0  1  0  1  0  0  1  0 -6645.0       6 ...
21   0   1   0   0   1   0   0   1  0  1  0  0  0  0  1  0 -6644.0       1 ...
23   0   1   0   0   0   1   0   1  0  1  0  1  0  0  1  0 -6639.0       1 ...
627  0   1   0   0   0   1   0   1  0  1  0  1  0  0

In [11]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q2 q3 q4 q5 q6 q7 q8 q9  energy num_oc. ...
4    0   1   0   0   1   0   0   1  0  1  0  1  0  0  1  1 -6690.0       1 ...
5    1   0   1   0   0   0   0   1  1  0  0  0  1  0  1  0 -6658.0       2 ...
422  1   1   0   0   0   0   0   1  0  1  0  0  0  0  1  0 -6658.0       1 ...
6    1   0   1   0   0   0   0   1  1  0  0  1  0  0  1  0 -6657.0       5 ...
7    0   1   0   0   0   1   0   1  0  1  0  0  0  0  1  0 -6644.0       2 ...
8    0   1   0   0   0   1   0   1  0  1  0  1  0  0  1  0 -6639.0       1 ...
9    0   0   1   0   0   0   0   1  0  1  0  1  0  0  1  0 -6631.0      10 ...
99   0   0   1   0   0   0   0   1  0  1  0  1  0  0  1  0 -6631.0       4 ...
10   1   0   1   0   1   0   0   1  1  0  0  0  1  0  1  0 -6614.0       5 ...
11   0   1   0   0   0   0   0   1  1  1  0  0  0  0  1  0 -6612.0       1 ...
12   0   0   1   0   0   0   0   1  1  0  0  0  1  0  1  0 -6612.0      16 ...
13   1   1   0   0   1   0   0   1  1  0  0  0  1  0

In [12]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q2 q3 q4 q5 q6 q7 q8 q9  energy num_oc. ...
18   0   1   0   0   0   0   0   1  0  1  0  1  0  0  1  1 -6710.0       3 ...
21   0   1   0   0   1   0   0   1  0  1  0  1  0  0  1  1 -6690.0       2 ...
24   1   1   0   0   1   0   0   1  1  0  0  1  0  0  1  1 -6676.0       1 ...
27   1   1   0   0   0   0   0   1  0  1  0  1  0  0  1  1 -6660.0       1 ...
28   1   0   1   0   0   0   0   1  1  0  0  0  1  0  1  0 -6658.0       1 ...
29   1   1   0   0   0   0   0   1  0  1  0  1  0  0  1  0 -6657.0       1 ...
30   0   1   0   0   1   0   0   1  0  1  0  1  0  0  1  0 -6655.0       4 ...
31   1   1   0   0   1   0   0   1  0  1  0  1  0  0  1  0 -6645.0       1 ...
32   0   1   0   0   0   1   0   1  0  1  0  0  0  0  1  0 -6644.0       1 ...
33   0   1   0   0   1   0   0   1  0  1  0  0  0  0  1  0 -6644.0       1 ...
35   0   1   0   0   0   1   0   1  0  1  0  1  0  0  1  0 -6639.0       5 ...
36   0   0   1   0   0   0   0   1  0  1  0  1  0  0

In [13]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q2 q3 q4 q5 q6 q7 q8 q9  energy num_oc. ...
2    0   1   0   0   0   0   0   1  0  1  0  1  0  0  1  1 -6710.0       2 ...
3    0   1   0   0   1   0   0   1  0  1  0  1  0  0  1  1 -6690.0       3 ...
4    1   1   0   0   1   0   0   1  1  0  0  1  0  0  1  1 -6676.0       2 ...
5    1   1   0   0   0   0   0   1  0  1  0  0  0  0  1  0 -6658.0       1 ...
6    1   1   0   0   0   0   0   1  0  1  0  1  0  0  1  0 -6657.0       2 ...
7    0   1   0   0   1   0   0   1  0  1  0  0  0  0  1  0 -6644.0       2 ...
8    0   1   0   0   0   0   0   1  0  1  0  1  0  0  1  0 -6631.0       1 ...
9    1   0   1   0   1   0   0   1  1  0  0  1  0  0  1  0 -6629.0       1 ...
10   0   1   0   0   0   0   0   1  0  1  0  0  1  0  1  1 -6629.0       3 ...
11   1   1   0   0   0   1   0   1  0  1  0  0  0  0  1  0 -6626.0       1 ...
12   0   1   0   0   1   0   0   1  0  1  0  0  0  0  1  1 -6625.0       1 ...
77   0   1   0   0   1   0   0   1  0  1  0  0  0  0

In [14]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q2 q3 q4 q5 q6 q7 q8 q9  energy num_oc. ...
209  0   1   0   0   1   0   0   1  0  1  0  1  0  0  1  1 -6690.0       1 ...
18   1   0   1   0   0   0   0   1  1  0  0  0  1  0  1  0 -6658.0       1 ...
22   0   0   1   0   0   0   0   1  0  1  0  1  0  0  1  0 -6631.0       2 ...
168  0   0   1   0   0   0   0   1  0  1  0  1  0  0  1  0 -6631.0       1 ...
23   1   0   1   0   1   0   0   1  1  0  0  1  0  0  1  0 -6629.0       2 ...
27   0   0   1   0   0   0   0   1  0  1  0  0  1  0  1  0 -6604.0       2 ...
28   0   0   1   0   1   0   0   1  1  0  0  0  1  0  1  0 -6604.0       2 ...
50   0   0   1   0   0   0   0   1  0  1  0  0  1  0  1  0 -6604.0       3 ...
222  0   0   0   0   1   0   0   1  1  1  0  0  0  0  1  1 -6593.0       1 ...
29   0   0   1   0   1   0   0   1  1  0  0  1  0  0  1  0 -6591.0       2 ...
31   0   0   1   0   0   0   0   1  1  0  0  0  1  0  1  1 -6581.0       3 ...
32   0   0   1   0   1   0   0   1  0  1  0  1  0  0

In [15]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q2 q3 q4 q5 q6 q7 q8 q9  energy num_oc. ...
25   0   1   0   0   1   0   0   1  0  1  0  1  0  0  1  1 -6690.0       1 ...
26   1   1   0   0   1   0   0   1  0  1  0  0  0  0  1  0 -6662.0       1 ...
27   1   1   0   0   0   0   0   1  1  0  0  1  0  0  1  1 -6660.0       3 ...
28   1   1   0   0   0   0   0   1  0  1  0  1  0  0  1  1 -6660.0       1 ...
29   1   0   1   0   0   0   0   1  1  0  0  0  1  0  1  0 -6658.0       1 ...
30   1   1   0   0   0   0   0   1  0  1  0  0  0  0  1  0 -6658.0       2 ...
31   1   1   0   0   0   0   0   1  0  1  0  1  0  0  1  0 -6657.0       1 ...
32   1   1   0   0   0   1   0   1  1  0  0  1  0  0  1  1 -6652.0       1 ...
33   1   1   0   0   1   0   0   1  0  1  0  1  0  0  1  0 -6645.0       2 ...
34   0   1   0   0   0   1   0   1  0  1  0  0  0  0  1  0 -6644.0       1 ...
35   1   0   1   0   1   0   0   1  1  0  0  1  0  0  1  0 -6629.0       1 ...
37   1   1   0   0   0   0   0   1  1  0  0  0  1  0

In [16]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q2 q3 q4 q5 q6 q7 q8 q9  energy num_oc. ...
10   0   1   0   0   0   0   0   1  0  1  0  1  0  0  1  1 -6710.0       2 ...
12   0   1   0   0   1   0   0   1  0  1  0  1  0  0  1  1 -6690.0       2 ...
14   1   1   0   0   1   0   0   1  1  0  0  1  0  0  1  1 -6676.0       4 ...
16   1   1   0   0   0   0   0   1  1  0  0  1  0  0  1  1 -6660.0       1 ...
17   1   1   0   0   0   0   0   1  0  1  0  1  0  0  1  1 -6660.0       1 ...
18   1   0   1   0   0   0   0   1  1  0  0  0  1  0  1  0 -6658.0       3 ...
19   1   0   1   0   0   0   0   1  1  0  0  1  0  0  1  0 -6657.0       1 ...
20   1   1   0   0   0   0   0   1  0  1  0  1  0  0  1  0 -6657.0       1 ...
306  1   0   1   0   0   0   0   1  1  0  0  1  0  0  1  0 -6657.0       1 ...
21   1   1   0   0   0   1   0   1  1  0  0  1  0  0  1  1 -6652.0       2 ...
22   0   1   0   0   1   0   0   1  0  1  0  0  0  0  1  0 -6644.0       1 ...
24   0   1   0   0   0   1   0   1  0  1  0  1  0  0

In [17]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q2 q3 q4 q5 q6 q7 q8 q9  energy num_oc. ...
0    0   1   0   0   0   0   0   1  0  1  0  1  0  0  1  1 -6710.0       2 ...
1    0   1   0   0   1   0   0   1  0  1  0  1  0  0  1  1 -6690.0       2 ...
3    1   1   0   0   1   0   0   1  1  0  0  1  0  0  1  1 -6676.0       9 ...
4    1   1   0   0   1   0   0   1  0  1  0  0  0  0  1  0 -6662.0       5 ...
5    1   1   0   0   0   0   0   1  0  1  0  1  0  0  1  1 -6660.0       2 ...
6    1   1   0   0   0   0   0   1  1  0  0  1  0  0  1  1 -6660.0       6 ...
7    1   1   0   0   0   0   0   1  0  1  0  0  0  0  1  0 -6658.0       3 ...
8    1   1   0   0   0   0   0   1  0  1  0  1  0  0  1  0 -6657.0       1 ...
282  1   1   0   0   0   0   0   1  0  1  0  1  0  0  1  0 -6657.0       1 ...
9    0   1   0   0   1   0   0   1  0  1  0  1  0  0  1  0 -6655.0       7 ...
10   1   1   0   0   0   1   0   1  1  0  0  1  0  0  1  1 -6652.0       3 ...
11   1   1   0   0   1   0   0   1  0  1  0  1  0  0